In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/SI630/

/content/drive/My Drive/SI630


In [ ]:
!pwd

/content/drive/My Drive/SI630


In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
!pip install torch

In [ ]:
import torch

In [ ]:
print ('Available devices ', torch.cuda.device_count())

Available devices  1


In [ ]:
!pip install simpletransformers

     |████████████████████████████████| 215kB 18.0MB/s 
     |████████████████████████████████| 3.3MB 14.5MB/s 
     |████████████████████████████████| 1.2MB 49.2MB/s 
     |████████████████████████████████| 122kB 58.0MB/s 
     |████████████████████████████████| 2.1MB 51.8MB/s 
     |████████████████████████████████| 51kB 8.7MB/s 
     |████████████████████████████████| 204kB 53.2MB/s 
     |████████████████████████████████| 81kB 11.2MB/s 
     |████████████████████████████████| 2.1MB 48.1MB/s 
     |████████████████████████████████| 8.2MB 46.6MB/s 
     |████████████████████████████████| 901kB 41.3MB/s 
     |████████████████████████████████| 112kB 58.6MB/s 
     |████████████████████████████████| 245kB 61.2MB/s 
     |████████████████████████████████| 163kB 58.1MB/s 
     |████████████████████████████████| 102kB 14.0MB/s 
     |████████████████████████████████| 133kB 58.3MB/s 
     |████████████████████████████████| 81kB 12.6MB/s 
     |████████████████████████████████| 4.2MB 53.0MB

In [ ]:
import json   
import logging
from simpletransformers.language_modeling import LanguageModelingModel
import pandas as pd

### Create the train, dev and test datasets

In [ ]:
with open('song-lyrics.train.jsonl', 'r') as json_file:
    json_list = list(json_file)
    
train_list = []
for json_str in json_list:
    result = json.loads(json_str)
    result['lyrics'] = result['lyrics'].replace('\n', '[XYZ] ')
    result['lyrics'] = result['lyrics'] + '<EOS>'
    train_list.append(result['lyrics'])

In [ ]:
len(train_list)

677272

In [ ]:
with open('train.txt', 'w') as fl:
    for line in train_list[0:100000]:
        fl.write(line+ '\n')

In [ ]:
with open('song-lyrics.dev.jsonl', 'r') as json_file:
    json_list = list(json_file)
    
dev_list = []
for json_str in json_list:
    result = json.loads(json_str)
    result['lyrics'] = result['lyrics'].replace('\n', '[XYZ] ')
    result['lyrics'] = result['lyrics'] + '<EOS>'
    dev_list.append(result['lyrics'])
    
with open('dev.txt', 'w') as fl:
    for line in dev_list:
        fl.write(line+'\n')

In [ ]:
len(dev_list)

84700

In [ ]:
with open('song-lyrics.test.jsonl', 'r') as json_file:
    json_list = list(json_file)
    
test_list = []
for json_str in json_list:
    result = json.loads(json_str)
    result['lyrics'] = result['lyrics'].replace('\n', '[XYZ] ')
    result['lyrics'] = result['lyrics'] + '<EOS>'
    test_list.append(result['lyrics'])
    
with open('test.txt', 'w') as fl:
    for line in test_list:
        fl.write(line+'\n')

In [ ]:
len(test_list)

84484

### Fine-tune the model

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "block_size": 64,
    "max_seq_length": 64,
    "learning_rate": 5e-6,
    "train_batch_size": 4, #8
    "gradient_accumulation_steps": 8,
    "num_train_epochs": 1,
    "mlm": False,
    "output_dir": f"outputs/fine-tuned/",
}

model = LanguageModelingModel("gpt2", "gpt2", args=train_args, use_cuda=True)

model.train_model(train_file = "train.txt", eval_file="dev.txt", output_dir = './tuned_model')


INFO:simpletransformers.language_modeling.language_modeling_utils: Creating features from dataset file at cache_dir/


Token indices sequence length is longer than the specified maximum sequence length for this model (1025 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1401 > 1024). Running this sequence through the model will result in indexing errors


INFO:simpletransformers.language_modeling.language_modeling_utils: Saving features into cached file cache_dir/gpt2_cached_lm_62_train50.txt


INFO:simpletransformers.language_modeling.language_modeling_model: Training started


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


INFO:simpletransformers.language_modeling.language_modeling_model: Training of gpt2 model complete. Saved to ./tuned_model50.


(12758, 2.8370193966430337)

In [ ]:
model = './tuned_model'

In [ ]:
perplexity = model.eval_model(eval_file = "dev1000.txt")
perplexity

INFO:simpletransformers.language_modeling.language_modeling_utils: Creating features from dataset file at cache_dir/


Token indices sequence length is longer than the specified maximum sequence length for this model (1262 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1549 > 1024). Running this sequence through the model will result in indexing errors


INFO:simpletransformers.language_modeling.language_modeling_utils: Saving features into cached file cache_dir/gpt2_cached_lm_62_dev1000.txt


INFO:simpletransformers.language_modeling.language_modeling_model:{'eval_loss': 2.712446319953895, 'perplexity': tensor(15.0661)}


{'eval_loss': 2.712446319953895, 'perplexity': tensor(15.0661)}

In [ ]:
perplexity_test = model.eval_model(eval_file = "test1000.txt")
perplexity_test

INFO:simpletransformers.language_modeling.language_modeling_utils: Creating features from dataset file at cache_dir/


Token indices sequence length is longer than the specified maximum sequence length for this model (1187 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1909 > 1024). Running this sequence through the model will result in indexing errors


INFO:simpletransformers.language_modeling.language_modeling_utils: Saving features into cached file cache_dir/gpt2_cached_lm_62_test1000.txt


INFO:simpletransformers.language_modeling.language_modeling_model:{'eval_loss': 2.7257728445127043, 'perplexity': tensor(15.2682)}


{'eval_loss': 2.7257728445127043, 'perplexity': tensor(15.2682)}

### Generate Lyrics

In [ ]:
from simpletransformers.language_generation import LanguageGenerationModel

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
prompts = ['My', 'The', 'One', 'When', 'If']

In [ ]:
generate_model = LanguageGenerationModel("gpt2", "tuned_model50", args={"max_length": 200},use_cuda=True)

In [ ]:
prompt_lyrics = {}
for p in prompts:
    lyrics = generate_model.generate(p, verbose=False)
    lyrics = lyrics[0]
    lyrics = lyrics.replace('[XYZ] ', '\n')
    lyrics = lyrics.replace('<EOS>', ' ')
    lyrics = lyrics.replace('\n\n', '\n')
    prompt_lyrics[p] = lyrics

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
prompt_lyrics

{'If': "If you're gonna fuck\nNow what?\nI'm gonna fuck you\nOh how you gonna fuck\nI'm gonna fuck you\nOh how you gonna fuck Yeah, I really loved him\nI really loved him\nAnd you love me, too\nI really loved him\nOh how you gonna fuck\nI've never loved him\nI'm a liar\nAnd you lie about you, too\nI really loved him\nOh how you gonna fuck\nI've never loved him\nI was told so\nAnd you lie about you\nI really loved him You gotta love me\nIt's just, it's just[XY",
 'My': "My own, my own, my own, my own\nSo I guess that's all that I'm worth\nI can tell it's all because I live in the heart of the town\nI was born on the hill, my own, my own\nNo, no, no, yeah, yeah\nI wanna live life for the people that know and love me, I want to be free, I wanna live life for the people that know and love me\nYou were a bad mornin' dream girl\nYou were the same\nYou were my sweet little dream girl, yeah\nYou were my sweet little dream girl\nAnd we never had love, I won't be your sweetest, I won't be you my

In [ ]:
with open("prompt_lyrics.json", "w") as outfile: 
    json.dump(prompt_lyrics, outfile)

In [ ]:
test_firstword = []
for l in test_list:
  firstword = l.split()[0]
  firstword = firstword.replace('[XYZ]','')
  test_firstword.append(firstword)

test_uniqueword = set(test_firstword)

In [ ]:
import random
test_uniqueword = random.sample(test_uniqueword, 550)

In [ ]:
len(test_uniqueword)

550

In [ ]:
lyrics_550_new = {}
for word in test_uniqueword:
    lyrics = generate_model.generate(word, verbose=False)
    lyrics = lyrics[0]
    lyrics = lyrics.replace('[XYZ] ', '\n')
    lyrics = lyrics.replace('<EOS>', ' ')
    lyrics = lyrics.replace('\n\n', '\n')
    lyrics_550_new[word] = lyrics

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [ ]:
with open("lyrics_550_new.json", "w") as outfile: 
    json.dump(lyrics_550_new, outfile)

In [ ]:
len(lyrics_550_new)

550